In [3]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input, decode_predictions
import numpy as np

import keras.backend as K
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

K.set_learning_phase(0) # set phase as testing

In [4]:
model = MobileNet(weights='imagenet')

# Make inference for an image

In [5]:
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02504013', 'Indian_elephant', 0.9265905), ('n02504458', 'African_elephant', 0.032345593), ('n01871265', 'tusker', 0.022741912)]


In [6]:
import tensorflow as tf
# Rename output
out_prefix = 'output'
OUT_NAMES = [None] * len(model.outputs)
OUT_TENSORS = [None] * len(model.outputs)
for i in range(len(model.outputs)):
    OUT_NAMES[i] = out_prefix + '_' + str(i)
    OUT_TENSORS = tf.identity(model.output[i], name=OUT_NAMES[i])
print(OUT_NAMES)
print(model.output)

['output_0']
Tensor("reshape_2/Reshape:0", shape=(?, 1000), dtype=float32)


# Save model as tensorflow model (.pb)


In [7]:
output_path = 'saved_tf_model/'
model_name = 'mobilenet_v1.pb'

In [9]:


sess = K.get_session()
# sess.graph_def

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), output_node_names=OUT_NAMES)
graph_io.write_graph(constant_graph, output_path, model_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', output_path + model_name)

INFO:tensorflow:Froze 137 variables.
Converted 137 variables to const ops.
saved the constant graph (ready for inference) at:  saved_tf_model/mobilenet_v1.pb
